In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train_over_under_sample')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 5
    models = ['bert-base-multilingual-cased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + '_over_under_sampled' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

...

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.4225066602230072
DEV ACC: 0.8065292475014698
DEV Precision: 0.6024096714572905
DEV Recall: 0.9592271925605259
DEV F1Score: 0.7251378339485434
BEST ACCURACY -->  DEV: 0.80653
BEST PRECISION -->  DEV: 0.60241
BEST RECALL -->  DEV: 0.95923
BEST F1SCORE -->  DEV: 0.72514
TIME PER EPOCH: 7.724245083332062

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.3714604675769806
DEV ACC: 0.8133450911228689
DEV Precision: 0.6160574434383957
DEV Recall: 0.9159423028470649
DEV F1Score: 0.7211464893106655
BEST ACCURACY -->  DEV: 0.81335
BEST PRECISION -->  DEV: 0.61606
BEST RECALL -->  DEV: 0.91594
BEST F1SCORE -->  DEV: 0.72115
TIME PER EPOCH: 8.31003706852595

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training epoch: 1
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIHED BATCH: 159 of 330
FINSIHED BATCH: 179 of 330
FINSIHED BATCH: 199 of 330
FINSIHED BATCH: 219 of 330
FINSIHED BATCH: 239 of 330
FINSIHED BATCH: 259 of 330
FINSIHED BATCH: 279 of 330
FINSIHED BATCH: 299 of 330
FINSIHED BATCH: 319 of 330
Validation loss per 100 evaluation steps: 0.7411448955535889
DEV ACC: 0.7934119635508524
DEV Precision: 0.6446108565156183
DEV Recall: 0.9099332325522798
DEV F1Score: 0.7389305780246364
BEST ACCURACY -->  DEV: 0.79341
BEST PRECISION -->  DEV: 0.64461
BEST RECALL -->  DEV: 0.90993
BEST F1SCORE -->  DEV: 0.73893
TIME PER EPOCH: 7.918842442830404

Training epoch: 2
FINSIHED BATCH: 19 of 330
FINSIHED BATCH: 39 of 330
FINSIHED BATCH: 59 of 330
FINSIHED BATCH: 79 of 330
FINSIHED BATCH: 99 of 330
FINSIHED BATCH: 119 of 330
FINSIHED BATCH: 139 of 330
FINSIH

In [6]:
print(best_dev_acc)

0.8374301881246327


In [7]:
print(best_epoch)

0


In [8]:
print(best_dev_precision)

0.6987070248975009


In [9]:
print(best_dev_recall)

0.9099332325522798


In [10]:
print(best_dev_f1score)

0.7389305780246364


In [11]:
print(overall_list_dev_acc)

[[0.8135288065843622, 0.8248456790123457, 0.839046884185773, 0.8289057907113463, 0.7940917107583775], [0.7956716637272193, 0.8299897119341564, 0.8228615520282188, 0.8041960611405057, 0.8208774250440917], [0.8065292475014698, 0.8323780129335685, 0.835133744855967, 0.8271421222810111, 0.8226778365667254], [0.8133450911228689, 0.7944040270429159, 0.8217408877131099, 0.8002278071722516, 0.7975639329805997], [0.7934119635508524, 0.8374301881246327, 0.8191688712522045, 0.7853835978835978, 0.8212815990593768]]


In [12]:
print(overall_list_dev_precision)

[[0.6245933519743043, 0.6044834354358164, 0.6865233972376827, 0.6659297052154195, 0.6214892691083166], [0.6879543736686593, 0.6305582865106675, 0.648954264430455, 0.6914462081128747, 0.6549869442726587], [0.6024096714572905, 0.6313577956435098, 0.6635785290547197, 0.6728469502279025, 0.6949483497102545], [0.6160574434383957, 0.6037452186261711, 0.6372844087129803, 0.6481607457797932, 0.6591109507776173], [0.6446108565156183, 0.6854938271604938, 0.6987070248975009, 0.6436793365364796, 0.6167159119540071]]


In [13]:
print(overall_list_dev_recall)

[[0.9226883345930965, 0.9486331569664901, 0.8168367346938774, 0.8084782060972537, 0.5805733067637827], [0.8364008566389519, 0.8920571932476694, 0.8551965230536657, 0.5589191232048375, 0.7604405735358116], [0.9592271925605259, 0.9169060216679262, 0.845660115898211, 0.7971936352888732, 0.6712459057697155], [0.9159423028470649, 0.944954648526077, 0.8370490620490619, 0.7809953274238988, 0.6786470143612998], [0.9099332325522798, 0.7304639364163171, 0.6892668178382463, 0.798875419113514, 0.7502960443436634]]


In [14]:
print(overall_list_dev_f1score)

[[0.7349000716405796, 0.7229848723796827, 0.7254826940659164, 0.708478929815359, 0.5798059259513405], [0.7398326307990171, 0.7277723124405042, 0.7204451013723233, 0.5914712634600591, 0.6839330342205174], [0.7251378339485434, 0.7309763312896143, 0.7259303558140483, 0.71520388632286, 0.6560480861601311], [0.7211464893106655, 0.7223131807518073, 0.7055855643586156, 0.682012756301419, 0.643736929878755], [0.7389305780246364, 0.6861787523662801, 0.6689061451813921, 0.6980074901686795, 0.6650575197567675]]


In [15]:
#The best model is 1